<a href="https://colab.research.google.com/github/tumurzakov/AnimateDiff/blob/main/Fine_tune_AnimateDiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Init
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Install
#!git clone -b v1.0 https://github.com/camenduru/animatediff
!git clone https://github.com/tumurzakov/AnimateDiff.git animatediff
!pip install omegaconf decord einops omegaconf safetensors diffusers[torch]==0.11.1 transformers xformers==0.0.20 triton==2.0.0 imageio[pyav] mediapy
!apt -y install -qq aria2
!rm -rf /content/animatediff/models/StableDiffusion
!git clone -b fp16 https://huggingface.co/runwayml/stable-diffusion-v1-5 /content/animatediff/models/StableDiffusion/
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v14.ckpt -d /content/animatediff/models/Motion_Module -o mm_sd_v14.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v15.ckpt -d /content/animatediff/models/Motion_Module -o mm_sd_v15.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/toonyou_beta3.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o toonyou_beta3.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v2.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o Deliberate_v2.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/CounterfeitV30_v30.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o CounterfeitV30_v30.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/FilmVelvia2.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o FilmVelvia2.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/Pyramid%20lora_Ghibli_n3.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o Pyramid%20lora_Ghibli_n3.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/TUSUN.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o TUSUN.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/lyriel_v16.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o lyriel_v16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/majicmixRealistic_v5Preview.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o majicmixRealistic_v5Preview.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/moonfilm_filmGrain10.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o moonfilm_filmGrain10.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/moonfilm_reality20.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o moonfilm_reality20.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/rcnzCartoon3d_v10.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o rcnzCartoon3d_v10.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/realisticVisionV40_v20Novae.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o realisticVisionV40_v20Novae.safetensors

Cloning into 'animatediff'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 268 (delta 113), reused 245 (delta 90), pack-reused 0
Receiving objects: 100% (268/268), 9.50 MiB | 27.86 MiB/s, done.
Resolving deltas: 100% (113/113), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.9/524.9 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 111.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━

The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,475 kB of archives.
After this operation, 5,959 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.15.0-1ubuntu0.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.35.0-1build1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.35.0-1build1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.35.0-1build1_amd64.deb ...
Unpacking aria2 (1.35.0-1build1) ...
Setting up libc-ares2:amd64 (1.15.0-1ubuntu0.3) ...
Setting up libaria2-0:amd64 (1.35.0-1build1) ...
Setting up aria2

In [ ]:
# @title Project

PROJECT = "city2" # @param{"type":"string"}

OUTPUT_DIR = "/content/drive/MyDrive/AI/models/AnimateDiff/%s" % PROJECT

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

[*] Weights will be saved at /content/drive/MyDrive/AI/models/AnimateDiff/city2


In [ ]:
#@title Make video
%cd /content/animatediff


from PIL import Image
import numpy as np
#!pip install imageio[pyav]

load_image_dir = "/content/drive/MyDrive/AI/video/videos/1" # @param{"type":"string"}
load_image_start = 1 # @param{"type":"integer"}
load_image_count = 120 #@param{"type":"integer"}
load_image_by = 4 #@param{"type":"integer"}
load_image_width = 512
load_image_height = 512
load_image_x_offset = 0
load_image_y_offset = 100
load_image_fps = 4

frames = []
for i in range(load_image_start, load_image_start + load_image_count, load_image_by):
  frame = Image.open('%s/%d.png' % (load_image_dir, i)).convert('L').convert('RGB')
  frame = frame.crop((
      frame.width/2-load_image_width/2+load_image_x_offset,
      frame.height/2-load_image_height/2+load_image_y_offset,
      frame.width/2+load_image_width/2+load_image_x_offset,
      frame.height/2+load_image_height/2+load_image_y_offset,
  ))
  frames.append(frame.resize((512,512)))

!mkdir -p outputs/frames
!mkdir -p data
for index, frame in enumerate(frames):
  frame.save("outputs/frames/%d.png" % index)

!rm -f data/{PROJECT}.mp4
!ffmpeg -r {load_image_fps} -i outputs/frames/%d.png -c:v libx264 -vf fps={load_image_fps} data/{PROJECT}.mp4

frames[0].save(
    f"data/{PROJECT}.webp",
    save_all=True,
    append_images=frames[1:],
    duration=int(len(frames)/load_image_fps) * 1000,
    loop=0,
)

import mediapy as media
media.show_video([np.array(img) for img in frames], fps=load_image_fps)

/content/animatediff
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab -

This browser does not support the video tag.


In [ ]:
#@title Write config
%cd /content/animatediff

MODEL_NAME = "/content/animatediff/models/StableDiffusion/"

config_yaml = f"""
pretrained_model_path: /content/animatediff/models/StableDiffusion/
output_dir: outputs/wind
train_data:
  video_path: data/{PROJECT}.mp4
  prompt: train prompt
  n_sample_frames: 16
  width: 512
  height: 512
  sample_start_idx: 0
  sample_frame_rate: 4
validation_data:
  prompts:
  - prompt
  video_length: 16
  width: 512
  height: 512
  num_inference_steps: 20
  guidance_scale: 12.5
  use_inv_latent: true
  num_inv_steps: 50
learning_rate: 3.0e-05
train_batch_size: 1
max_train_steps: 300
checkpointing_steps: 1000
validation_steps: 100
trainable_modules:
- attn1.to_q
- attn2.to_q
- attn_temp
seed: 33
mixed_precision: fp16
use_8bit_adam: false
gradient_checkpointing: true
enable_xformers_memory_efficient_attention: true
"""

with open(f"configs/{PROJECT}.yaml", 'w') as f:
  f.write(config_yaml)

/content/animatediff


In [ ]:
#@markdown Train config
%cd /content/animatediff

from omegaconf import OmegaConf

#OUTPUT_DIR = "outputs/city" #@param{"type":"string"}
CONFIG_NAME = f"configs/{PROJECT}.yaml" #@param {type:"string"}
MOTION_MODULE_NAME="/content/drive/MyDrive/AI/models/AnimateDiff/mm_sd_v15.ckpt" #@param{"type":"string"}

train_video_path = f"data/{PROJECT}.mp4" #@param {type:"string"}
train_prompt = "flying over the skyscrapers of the metropolis" #@param {type:"string"}
video_length = 16 #@param {type:"number"}
width = 512 #@param {type:"number"}
height = 512 #@param {type:"number"}
learning_rate = 3e-5 #@param {type:"number"}
train_steps = 1 #@param {type:"number"}
validation_steps = 1 #@param {type:"number"}

config = {
  "pretrained_model_path": MODEL_NAME,
  "motion_module": MOTION_MODULE_NAME,
  "output_dir": OUTPUT_DIR,
  "train_data": {
    "video_path": train_video_path,
    "prompt": train_prompt,
    "n_sample_frames": video_length,
    "width": width,
    "height": height,
    "sample_start_idx": 0,
    "sample_frame_rate": 1,
  },
  "validation_data": {
    "prompts": [
      f"{train_prompt}",
      f"{train_prompt}, intricate details, art, cyberpunk, scifi, digital painting, cinematic, hdr 8k",
      #"a young woman on the wind, red dress, looking out the window at metropolis landscape, intricate details, art, digital painting, cinematic, hdr 8k",
    ],
    "video_length": video_length,
    "width": width,
    "height": height,
    "num_inference_steps": 20,
    "guidance_scale": 12.5,
    "use_inv_latent": True,
    "num_inv_steps": 50,
  },
  "learning_rate": learning_rate,
  "train_batch_size": 1,
  "max_train_steps": train_steps,
  "checkpointing_steps": 100,
  "validation_steps": validation_steps,
  "trainable_modules": [
    "to_q",
  ],
  "seed": 33,
  "mixed_precision": "fp16",
  "use_8bit_adam": False,
  "gradient_checkpointing": True,
  "enable_xformers_memory_efficient_attention": True,
}

OmegaConf.save(config, CONFIG_NAME)

/content/animatediff


In [ ]:
#@title Train
%cd /content/animatediff

!git reset --hard
!git pull origin main
print(CONFIG_NAME)
!accelerate launch train.py --config=$CONFIG_NAME
#!python train.py --config=$CONFIG_NAME

/content/animatediff
HEAD is now at e5b502b save sampels by idx
From https://github.com/tumurzakov/AnimateDiff
 * branch            main       -> FETCH_HEAD
Already up to date.
configs/city2.yaml
2023-07-16 11:03:02.338060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2023-07-16 11:03:08.999835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
07/16/2023 11:03:14 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'vari

In [ ]:
%cd /content/animatediff

# !python -m scripts.animate --config /content/animatediff/configs/prompts/city2.yaml --pretrained_model_path /content/animatediff/models/StableDiffusion --L 16 --W 512 --H 512